In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
pd.set_option('display.max_columns', None)

In [5]:
mlb = pd.read_csv('MLB.txt', delimiter = "\t")

In [6]:
mlb.summary()

AttributeError: 'DataFrame' object has no attribute 'summary'